# API em URL Público

Criação de uma conta no site da https://ngrok.com/

Pós-criação, copiar a API_KEY do link:
https://dashboard.ngrok.com/get-started/your-authtoken

Criar um dominio no Ngrok
https://dashboard.ngrok.com/cloud-edge/domains

# Variáveis de ambiente

É necessário criar duas secrets aqui no colab (Ícone de chave na lateral direita do menu)

As chaves necessárias são:

- NGROK_TOKEN
- NGROK_DOMAIN



In [9]:
from google.colab import userdata

# Token de acesso ao Ngrok
NGROK_API_KEY = userdata.get('NGROK_TOKEN')

# Dominio criado no Ngrok
NGROK_DOMAIN = userdata.get('NGROK_DOMAIN')

# Escolha do modelo Ollama para uso
MODEL_NAME_OLLAMA = 'llama2'


# Instalação de Ollama no Google Colab Server

In [10]:
# Download and install ollama to the system
!curl https://ollama.ai/install.sh | sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0>>> Downloading ollama...
100 10044    0 10044    0     0  33452      0 --:--:-- --:--:-- --:--:-- 33368
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


# Instalação de pacotes Python

In [11]:
!pip install -qU aiohttp pyngrok

# Execução e criação de API em URL Pública

In [ ]:
# execução e criação da API
import os
import asyncio

# Set LD_LIBRARY_PATH so the system NVIDIA library
os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})

os.environ['OLLAMA_HOST'] = '0.0.0.0'


async def run_process(cmd):
  print('>>> starting', *cmd)
  p = await asyncio.subprocess.create_subprocess_exec(
      *cmd,
      stdout=asyncio.subprocess.PIPE,
      stderr=asyncio.subprocess.PIPE,
  )

  async def pipe(lines):
    async for line in lines:
      print(line.strip().decode('utf-8'))

  await asyncio.gather(
      pipe(p.stdout),
      pipe(p.stderr),
  )

#register an account at ngrok.com and create an authtoken and place it here
await asyncio.gather(
    run_process(['ngrok', 'config', 'add-authtoken', NGROK_API_KEY])
)

await asyncio.gather(
    run_process(['ollama', 'serve']),
    run_process(['ollama', 'run', MODEL_NAME_OLLAMA]),
    run_process(['ngrok', 'http', f'--domain={NGROK_DOMAIN}', '--log', 'stderr', '11434']),
)